In [43]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

df = pd.read_csv('df.csv')
df.head()

,Unnamed: 0,type_of_nucleic_acid,aptamer_sequence,sequence_length,gc_content,kd_(nm),type_of_the_buffer,ph,filtered_sequence,mol_wt,num_rotatable_bonds,tpsa,mol_log_p
0,1,12,GAAUUGUGGUGUUGGCUCCCUAUAGUGAGUCGUAUUAAUAUUCCUU...,113,0.336283,4.8,0,7.466744,MKEFYISIETVGNNIVERYIDENGKERTREVEYLPTMFRHCKEESK...,134278.328,714.0,54936.71,-838.74460
1,2,12,GAAUUGUGGUGUUGGCUCCCUAUAGUGAGUCGUAUUAAUAUUCCUU...,113,0.353982,4.8,0,7.466744,MKEFYISIETVGNNIVERYIDENGKERTREVEYLPTMFRHCKEESK...,34932.516,713.0,14449.86,-254.95599
2,3,11,GGTTGGTGTGGTTGG,15,0.600000,112.5,3,7.400000,TSEDHFQPFFNEKTFGAGEADCGLRPLFEKKQVQDQTEKELFESYI...,73107.795,730.0,30882.71,-636.91317
3,4,11,GGTTGG,6,0.666667,112.5,3,7.400000,TSEDHFQPFFNEKTFGAGEADCGLRPLFEKKQVQDQTEKELFESYI...,69525.328,704.0,28032.07,-543.13314
4,5,12,GGGAGCAUCAGACUUUUAAUCUGACAAUCAAGAAUUCCGUUUUCAG...,96,0.333333,5.0,3,7.700000,PISPIETVPVKLKPGMDGPKVKQWPLTEEKIKALVEICTEMEKEGK...,74526.975,726.0,30647.61,-663.15408


In [7]:
df['mol_wt'].unique()

array([134278.328     ,  34932.516     ,  73107.795     ,  69525.328     ,
        74526.975     ,  11052.178     ,  44940.539     ,  19938.066     ,
        47166.533     ,  42273.264     , 109735.536     ,  69457.366     ,
       163280.03100001,  57511.675     ,   7725.818     ,  62547.704     ,
        56528.333     ,  42813.299     ,  38248.939     ,  97170.731     ,
        20608.113     ,  88885.085     ,  38238.927     ,  52682.49      ,
        20713.165     ,  35149.246     ,   9769.343     ,  57100.996     ,
        44851.924     ,  68407.309     ,             nan,  17279.952     ,
       103610.335     , 162042.68200001,   1070.299     ,  16028.531     ,
        47777.632     ,  76751.307     ,  35609.101     ,  23501.421     ,
        28450.903     ,  23614.036     ,  32329.98      ,  47065.86      ,
        13714.625     ,  22086.628     ,  30640.235     ,  87028.584     ,
        38177.801     , 133995.162     ,  49591.304     ,  45060.233     ,
        64368.209     ,  

В колонке присутвует nan, удалим такие колонки.

In [45]:
df.dropna(subset=['mol_wt'], inplace=True)
df.dropna(subset=['num_rotatable_bonds'], inplace=True)
df.dropna(subset=['tpsa'], inplace=True)
df.dropna(subset=['mol_log_p'], inplace=True)

## Регресионное моделирование

In [39]:
!pip install catboost

In [46]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 641 entries, 0 to 649
Data columns (total 13 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Unnamed: 0            641 non-null    int64  
 1   type_of_nucleic_acid  641 non-null    int64  
 2   aptamer_sequence      641 non-null    object 
 3   sequence_length       641 non-null    int64  
 4   gc_content            641 non-null    float64
 5   kd_(nm)               641 non-null    float64
 6   type_of_the_buffer    641 non-null    int64  
 7   ph                    641 non-null    float64
 8   filtered_sequence     641 non-null    object 
 9   mol_wt                641 non-null    float64
 10  num_rotatable_bonds   641 non-null    float64
 11  tpsa                  641 non-null    float64
 12  mol_log_p             641 non-null    float64
dtypes: float64(7), int64(4), object(2)
memory usage: 70.1+ KB


In [49]:
cat = [
    'type_of_nucleic_acid',
    'mol_wt',
    'num_rotatable_bonds',
    'tpsa',
    'mol_log_p'
]
   


In [52]:
from catboost import CatBoostRegressor, Pool, cv

param_grid = {
    'iterations': [100, 500, 1000],
    'learning_rate': [0.01, 0.05, 0.1],
    'depth': [4, 6, 8],
}

df['gc_content'] = df['gc_content'].astype(str)
df['kd_(nm)'] = df['kd_(nm)'].astype(str)
df['ph'] = df['ph'].astype(str)
df['mol_wt'] = df['mol_wt'].astype(str)
df['num_rotatable_bonds'] = df['num_rotatable_bonds'].astype(str)
df['tpsa'] = df['tpsa'].astype(str)
df['mol_log_p'] = df['mol_log_p'].astype(str)

model = CatBoostRegressor(verbose=False)

train_pool = Pool(data=df.drop({'kd_(nm)', 'aptamer_sequence', 'filtered_sequence'}, axis=1), label=df['kd_(nm)'], cat_features=cat)

grid_search_result = model.grid_search(param_grid, train_pool, cv=3, plot=True)

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))


bestTest = 119.8302205
bestIteration = 99

0:	loss: 119.8302205	best: 119.8302205 (0)	total: 3.26s	remaining: 1m 24s

bestTest = 110.8329634
bestIteration = 98

1:	loss: 110.8329634	best: 110.8329634 (1)	total: 5.68s	remaining: 1m 11s

bestTest = 109.1205833
bestIteration = 81

2:	loss: 109.1205833	best: 109.1205833 (2)	total: 8.56s	remaining: 1m 8s

bestTest = 112.0131011
bestIteration = 489

3:	loss: 112.0131011	best: 109.1205833 (2)	total: 20.2s	remaining: 1m 56s

bestTest = 103.0272742
bestIteration = 499

4:	loss: 103.0272742	best: 103.0272742 (4)	total: 32.7s	remaining: 2m 24s

bestTest = 102.979766
bestIteration = 471

5:	loss: 102.9797660	best: 102.9797660 (5)	total: 45.2s	remaining: 2m 38s

bestTest = 109.6691468
bestIteration = 998

6:	loss: 109.6691468	best: 102.9797660 (5)	total: 1m 8s	remaining: 3m 16s

bestTest = 102.3921598
bestIteration = 904

7:	loss: 102.3921598	best: 102.3921598 (7)	total: 1m 36s	remaining: 3m 48s

bestTest = 102.076558
bestIteration = 863

8:	loss:

In [53]:
feature_importance = model.get_feature_importance(type="FeatureImportance")

feature_names = df.drop({'kd_(nm)', 'aptamer_sequence', 'filtered_sequence'}, axis=1).columns

feature_importance_dict = dict(zip(feature_names, feature_importance))

sorted_feature_importance = sorted(feature_importance_dict.items(), key=lambda x: x[1], reverse=True)

for feature, importance in sorted_feature_importance:
    print(f"Feature: {feature}, Importance: {importance}")

Feature: Unnamed: 0, Importance: 33.48490017960948
Feature: gc_content, Importance: 18.334449014673183
Feature: sequence_length, Importance: 14.397731603346159
Feature: type_of_nucleic_acid, Importance: 10.139522864871264
Feature: ph, Importance: 10.03142910650108
Feature: type_of_the_buffer, Importance: 6.3881364117381905
Feature: num_rotatable_bonds, Importance: 4.233523717807383
Feature: mol_log_p, Importance: 1.55414293393984
Feature: mol_wt, Importance: 1.0113386351851719
Feature: tpsa, Importance: 0.42482553232829917


In [ ]:
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error


mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)

print(f'MSE: {mse}, R^2: {r2}, MAE: {mae}')

In [37]:
from sklearn.linear_model import LinearRegression

# Обучение модели
model = LinearRegression()
model.fit(X_train, y_train)

# Оценка модели
y_pred = model.predict(X_val)



In [38]:
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error


mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)

print(f'MSE: {mse}, R^2: {r2}, MAE: {mae}')



MSE: 8204.875963653845, R^2: 0.025232275212453636, MAE: 63.02243578093862
